In [24]:
import pandas as pd
from stage import fullPrint
from datetime import datetime, timedelta
import yahoo_fin.stock_info as yf
import numpy as np
import swifter
import os

nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)

PEAK = 13
allList = []
nasdaqList = yf.tickers_nasdaq()
allList = (nasdaqList + nyseList)

SectorDict = {
    "Energy":"XLE",
    "Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Defensive":"XLP",
    "Healthcare":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrials":"XLI",
    "Utilities":"XLU",
    "Basic Materials":"XLB",
    "Financial Services":"XLF",
    "Real Estate":"XLRE"}
spList = yf.tickers_sp500()
spList = set(spList)
dictionary = {}
i = 0
allLen = str(len(nasdaqList))
stockDataTickerList = []
now = datetime.now()

In [11]:
for ticker in nasdaqList:
    try:
        df = yf.get_data(ticker, start_date="1995-01-06",end_date = now, index_as_date = True)
        dfBuy = df
        df['Dates'] = pd.to_datetime(df.index)
        df = df[df['Dates'].dt.weekday == 4]
        df = df.drop('Dates', axis = 1)
        df.to_pickle('stockData/nysenasdaq/'+ticker+'.pkl')
        industry = yf.get_company_info(ticker).at['sector','Value']
        dictionary[ticker] = SectorDict[industry]
        dfBuy['Dates'] = pd.to_datetime(dfBuy.index)
        dfBuy = dfBuy[dfBuy['Dates'].dt.weekday == 0]
        dfBuy = dfBuy.drop('Dates', axis = 1)
        dfBuy.to_pickle('stockData/nysenasdaq/'+ticker+'Buy.pkl')
        stockDataTickerList.append(ticker)
    except:
        print(ticker+" not found")
    i = i+1
    print(str(i)+'/'+allLen)
import pickle
try:
    sectorDict = open('stockData/nasdaq.pkl', 'wb')
    pickle.dump(dictionary, sectorDict)
    sectorDict.close()
  
except:
    print("Something went wrong")

1/5586
2/5586
3/5586
AACIW not found
4/5586
5/5586
AADR not found
6/5586
7/5586
8/5586
9/5586
10/5586
11/5586
12/5586
13/5586
AAXJ not found
14/5586
15/5586
16/5586
17/5586
18/5586
19/5586
20/5586
21/5586
22/5586
23/5586
24/5586
25/5586
26/5586
27/5586
28/5586
29/5586
30/5586
31/5586
32/5586
ACAHW not found
33/5586
34/5586
35/5586
36/5586
37/5586
ACBAW not found
38/5586
39/5586
40/5586
41/5586
42/5586
43/5586
44/5586
ACEVW not found
45/5586
46/5586
47/5586
48/5586
49/5586
50/5586
51/5586
52/5586
53/5586
ACKIT not found
54/5586
ACKIU not found
55/5586
ACKIW not found
56/5586
57/5586
58/5586
59/5586
60/5586
61/5586
62/5586
63/5586
ACQRW not found
64/5586
65/5586
66/5586
67/5586
68/5586
69/5586
70/5586
ACTDW not found
71/5586
72/5586
73/5586
ACWI not found
74/5586
ACWX not found
75/5586
76/5586
77/5586
78/5586
79/5586
80/5586
81/5586
82/5586
83/5586
84/5586
ADERW not found
85/5586
86/5586
87/5586
88/5586
89/5586
ADILW not found
90/5586
91/5586
92/5586
93/5586
ADNWW not found
94/5586
95/55

In [26]:
weights = np.arange(1,31)
sumWeights = np.sum(weights)
spList = yf.tickers_sp500()
def calculateSlopeOfTicker(df,index):
    if index < 31:
        return 0
    else:
        return df.iat[index,4]/df.iat[index-1,4]

def product(df,index):
    if index < 30:
        return 0
    else:
        return df.iat[index,0]/df.iat[index-30,0]
def calculatePeak(df,index, diff, col):
    if index == 0:
        return df.iat[index,0]
    if index < diff:
        return max(df.iat[index-1,col],df.iat[index,0])


In [27]:
spDf = yf.get_data('^GSPC', start_date="1995-01-06",end_date = now, index_as_date = True)
spDf['close'] = spDf['close'].rolling(window=5).apply(lambda x: np.average(x))
spDf['Dates'] = pd.to_datetime(spDf.index)
spDf = spDf[spDf['Dates'].dt.weekday == 4]
spDf = spDf.drop('Dates', axis = 1)
spDf['percent'] = spDf.swifter.apply(lambda x: product(spDf,spDf.index.get_loc(x.name)),axis=1)
spDf['WMA30'] = spDf['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
spDf['WMA30Slope'] = spDf.swifter.apply(lambda x: calculateSlopeOfTicker(spDf,spDf.index.get_loc(x.name)),axis=1)
spDf.to_pickle("stockData/Spy.pkl")

Pandas Apply: 100%|██████████| 1368/1368 [00:00<00:00, 26322.88it/s]


In [14]:
for ticker in stockDataTickerList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        if(stockDf['close'].isnull().values.any()):
            os.remove('stockData/nyseNasdaq/'+ticker+'.pkl')
            continue
        stockDf['fiveYearHigh'] = stockDf['close'].rolling(window=260).max().shift(1).fillna(0)  
        stockDf['WMA30'] = stockDf['close'].rolling(window=30).apply(lambda x: np.sum(weights*x)/sumWeights)
        stockDf['WMA30Slope'] = stockDf.swifter.apply(lambda x: calculateSlopeOfTicker(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['volumePerc'] = stockDf['volume'].pct_change()
        stockDf['percent'] = stockDf.swifter.apply(lambda x: product(stockDf,stockDf.index.get_loc(x.name)),axis=1)
        stockDf['RS'] = stockDf['percent'] - spDf['percent']
        #stockDf = stockDf.dropna()
        stockDf['peak'] = 0
        stockDf['trough'] = 0
        stockDf['support'] = 0
        stockDf['initialSupport'] = 0
        for index, element in stockDf.iterrows():
            numIndex =stockDf.index.get_loc(index)
            if numIndex < 260:
                stockDf.iat[numIndex, PEAK] = calculatePeak(stockDf, numIndex, 260, 3)
        stockDf['Stage'] = ""
        stockDf['secondBuy'] = False
        stockDf['resistance'] = 0
        stockDf['shortPeak'] = 0
        stockDf['shortTrough'] = 0
        print(ticker)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass

Pandas Apply: 100%|██████████| 1368/1368 [00:00<00:00, 22312.47it/s]


AACG


Pandas Apply: 100%|██████████| 1368/1368 [00:00<00:00, 24196.99it/s]


AACI


Pandas Apply: 100%|██████████| 1368/1368 [00:00<00:00, 24410.70it/s]

...

ZY


Pandas Apply: 100%|██████████| 1368/1368 [00:00<00:00, 26209.49it/s]


ZYNE


Pandas Apply: 100%|██████████| 1368/1368 [00:00<00:00, 27514.18it/s]

ZYXI


In [14]:
transactionTemplate = yf.get_data('AAPL', start_date="1995-01-02",end_date= now, index_as_date = True, interval="1wk").drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate.to_pickle("transactionTemplate.pkl")

In [15]:
for ticker in allList:
    try:
        stockDf = pd.read_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
        stockDf = stockDf.drop(['open','high','low','adjclose'],axis=1)
        stockDf.to_pickle('stockData/nyseNasdaq/'+ticker+'.pkl')
    except:
        pass

In [20]:
now = datetime.now()
monday = now - timedelta(days = now.weekday())
now = now.strftime('%Y-%m-%d')
df = yf.get_data(ticker, start_date="1995-01-06",end_date = now, index_as_date = True)
df['Dates'] = pd.to_datetime(df.index)
df = df[df['Dates'].dt.weekday == 4]
df = df.drop('Dates', axis = 1)
df

,open,high,low,close,adjclose,volume,ticker
2017-04-28,13.50,14.25,13.00,13.00,13.00,953900,ZYME
2017-05-05,13.16,13.20,13.00,13.05,13.05,20500,ZYME
2017-05-12,13.09,13.12,13.00,13.00,13.00,5400,ZYME
2017-05-19,13.03,13.05,13.00,13.04,13.04,7400,ZYME
2017-05-26,13.04,13.04,12.50,12.59,12.59,16500,ZYME
...,...,...,...,...,...,...,...
2022-01-28,8.22,8.50,7.85,8.45,8.45,2754000,ZYME
2022-02-04,7.47,8.05,7.41,7.95,7.95,1132100,ZYME
2022-02-11,7.68,8.00,7.42,7.58,7.58,708700,ZYME
2022-02-18,7.06,7.38,7.04,7.18,7.18,633800,ZYME


In [18]:
with open('stockData/tickerList.pkl', 'wb') as f:
    pickle.dump(stockDataTickerList, f)

In [4]:
transactionTemplate = yf.get_data('AAPL', start_date="1995-01-06",end_date= now, index_as_date = True).drop(['open','high','low','close','adjclose','volume','ticker'],axis=1)
transactionTemplate['Dates'] = pd.to_datetime(transactionTemplate.index)
transactionTemplate = transactionTemplate[transactionTemplate['Dates'].dt.weekday == 4]
transactionTemplate = transactionTemplate.drop('Dates', axis = 1)
transactionTemplate.to_pickle("transactionTemplate.pkl")